In [3]:
import bs4 as bs
import urllib.request
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

url = 'https://httpbin.org/user-agent'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
request = urllib.request.Request(url,headers={'User-Agent': user_agent})
info_list = []
driver = webdriver.Chrome()

#prompt company
search = input('Company: ')
#prompt how many pages you want to scrape (each page contains 11 job links)
rows = int(input('How many pages: '))

#append search variable to link
link = 'https://www.simplyhired.com/search?q=' + search + '&pn='

#Scraper function
def Scraper(link, rows):
    
    #pass variables
    web = link
    
    #declare empty lists for hold
    link_list = []
    temp = []
    final_rows = []
    
    #First traversal: GET ALL LINKS FOF JOBS
    for i in range(1, rows):
        idx = i
        idx = str(i)
        web_link = web + idx
        driver.get(web_link)
        
        elems = driver.find_elements_by_xpath("//a[@href]")
        for elem in elems:
            temp.append(elem.get_attribute("href"))

      
    # store all job links to a list / remove all duplicates in the list          
    link_list = [ x for x in temp if "/job"  in x ]
    link_list = list(dict.fromkeys(link_list))
    print(link_list, len(link_list))
    
    #Second Traversal: GET ALL INFORMATION ON EACH JOB LINKS / GET TEXT FROM HTML TAGS
    for idx,val in enumerate(link_list):
        job_title = ''
        job_loc = ''
        job_type = ''
        job_salary = ''
        jt = []
        jl = []
        source = urllib.request.urlopen(link_list[idx]).read()
        soup = bs.BeautifulSoup(source, features='lxml')
        
        # ERROR HANDLING: if it provides an error, move on to next traversal, job_Link ignored
        try:
            company = soup.find('span', class_ = 'company').text
            job_title = soup.find('h1').text
            job_loc = soup.find('span', class_ = 'location').text
            salary = soup.find_all('span', {'class': 'info-unit'})
            s = []
            for i in salary:
                i = i.text
                i = i.replace('Estimated: ', "")
                s.append(i)
            s = [ x for x in s if "days ago" not in x ]
            s = [ x for x in s if "day ago" not in x ]
            s = [ x for x in s if "hours ago" not in x ]
            s = [ x for x in s if "hour ago" not in x ]
            job_type = s[0] 
            job_salary = s[1]
            
            print('Company: ', company)
            print('Job-title: ', job_title)
            print('Job-location: ',job_loc)
            print('Job Type: ', job_type)
            print('Estimated Salary: ', job_salary)
            
            header = soup.find_all('div', {'class': 'viewjob-entities'})
            h = []
            for i in header:
                i = i.text
                h.append(i)
            h = ''.join(h)
            h = re.sub('([A-Z])', r' \1', h)

            
            other_deets = soup.find_all('div', class_ = 'viewjob-description ViewJob-description')
            od = []
            for i in other_deets:
                i = i.text
                od.append(i)
            od = ''.join(od)
            od = re.sub('([A-Z])', r' \1', od)

            
            #compute percentages
            percentage = round(float(((idx+1)/len(link_list))*100), 2)
            print('Status: ', idx+1, '/', len(link_list), ' ', percentage, '%')
            print('\n')
            res = [job_title, job_loc, job_type, job_salary, h, od, company]
            final_rows.append(res)
            time.sleep(5)
            
        except:
            print('Empty. On to the next Traversal\n')
            continue
    
    # convert list to a Dataframe -------> convert Dataframe to CSV 
    print(final_rows)      
    df = pd.DataFrame(final_rows, columns = ['Job Title', 'Location', 'Job Type', 'Estimated Salary', 'Education/Skills/Benefits', 'Other Details', 'Company'])
    df = df[df['Company'] == search]
    tempstring = search + '-SimplyHired.csv' 
    df.to_csv(tempstring)
    print(df)
    print('Rows successful: ', len(df), '/', len(final_rows))
    print('\n\n CSV CREATED!')

Scraper(link,rows)

Company: Pearson
How many pages: 5
['https://www.simplyhired.com/job/YMXm_fG2A0q_1ruiVntV4_0dLeSkzJxJP1oM2wqrjYNQ86ZSEgptiQ?q=Pearson', 'https://www.simplyhired.com/job/1ILhbdiX1Uwv15LnWFphvXYtkK-XLYX2qdo9zxBKkCSC4XTgxtPO3g?q=Pearson', 'https://www.simplyhired.com/job/I5yBWmxguWrj7ExoZjrGgOozOcZl_QonFxogZ8NDJQl0OWVhrRnLEg?q=Pearson', 'https://www.simplyhired.com/job/ZNRZFxvgaEys5gygwl6djmeKF69LLXumPq20uX9UXJAo8vSKzbT1nQ?q=Pearson', 'https://www.simplyhired.com/job/BzqrniK-94cK9a-Aqaj1xcjYCGS1Rd0dDMtHigzbR7sHOpU5FlO37w?q=Pearson', 'https://www.simplyhired.com/job/NjiaFpUJgBjTdqXqN4A6y_coLym5qhCwlMiSclGUrBaw7hu3fPTCxQ?q=Pearson', 'https://www.simplyhired.com/job/ExeeUiTMgnUeufBA8G4e_FNgWPjdt6Og9iKuyAQ5VF96YRThl9jtDg?q=Pearson', 'https://www.simplyhired.com/job/WSwYSvGpv96ZLBCT2fu-JEAx12ExCfWjlEUiR0FkNF5K9xeToSLM8A?q=Pearson', 'https://www.simplyhired.com/job/gmozSfsIRBMfsKfC3Nr8LIj28ZRDl9eNr_uwL2MSVyIxYnJm2DSRtQ?q=Pearson', 'https://www.simplyhired.com/job/8UuMHVLo76NhmlFWfKGENScI-MdYgPE

Company:  Pearson
Job-title:  Channel Quality Specialist (Bloomington, MN)
Job-location:  Bloomington, MN
Job Type:  Full-time
Estimated Salary:  $51,000 - $64,000 a year
Status:  33 / 44   75.0 %


Company:  Pearson
Job-title:  Content Specialist - Mathematics
Job-location:  Iowa City, IA
Job Type:  Full-time
Estimated Salary:  $53,000 - $71,000 a year
Status:  34 / 44   77.27 %


Empty. On to the next Traversal

Company:  Pearson
Job-title:  Senior Customer Service Representative- Hadley, MA
Job-location:  Hadley, MA
Job Type:  Full-time
Estimated Salary:  $29,000 - $40,000 a year
Status:  36 / 44   81.82 %


Company:  Pearson
Job-title:  Associate Project Manager
Job-location:  Boston, MA
Job Type:  Full-time
Estimated Salary:  $70,000 - $92,000 a year
Status:  37 / 44   84.09 %


Company:  Pearson
Job-title:  Content Specialist - ELA
Job-location:  Iowa City, IA
Job Type:  Full-time
Estimated Salary:  $35,000 - $48,000 a year
Status:  38 / 44   86.36 %


Company:  Pearson
Job-title